<a href="https://colab.research.google.com/github/tx1103mark/tweet-sentiment/blob/master/TPUs_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPUs in Colab&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>
In this example, we'll work through training a model to classify images of
flowers on Google's lightning-fast Cloud TPUs. Our model will take as input a photo of a flower and return whether it is a daisy, dandelion, rose, sunflower, or tulip.

We use the Keras framework, new to TPUs in TF 2.1.0. Adapted from [this notebook](https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/courses/fast-and-lean-data-science/07_Keras_Flowers_TPU_xception_fine_tuned_best.ipynb) by [Martin Gorner](https://twitter.com/martin_gorner).

#### License

Copyright 2019-2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


---


This is not an official Google product but sample code provided for an educational purpose.


## Enabling and testing the TPU

First, you'll need to enable TPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select TPU from the Hardware Accelerator drop-down

Next, we'll check that we can connect to the TPU:

#Data process

In [ ]:
From fb1d9f2eac9731920a485cc30b6b7d64d0110fc7 Mon Sep 17 00:00:00 2001
From: guohongzilong <guohongzilong@huawei.com>
Date: Tue, 16 Mar 2021 20:47:38 +0800
Subject: [PATCH] fix unmond bug

---
 mindspore/lite/tools/anf_exporter/anf_exporter.cc  |  7 +--
 .../optimizer/graph/redundant_op_remove_pass.cc    | 50 +++++++++++++++++++++-
 .../optimizer/graph/redundant_op_remove_pass.h     |  1 +
 3 files changed, 53 insertions(+), 5 deletions(-)

diff --git a/mindspore/lite/tools/anf_exporter/anf_exporter.cc b/mindspore/lite/tools/anf_exporter/anf_exporter.cc
index 7a7207c..4d6e226 100644
--- a/mindspore/lite/tools/anf_exporter/anf_exporter.cc
+++ b/mindspore/lite/tools/anf_exporter/anf_exporter.cc
@@ -135,6 +135,7 @@ void AnfExporter::RemoveIfDepend(const CNodePtr &cnode) {
                                            opt::CheckPrimitiveType(depend_node, prim::kPrimControlDepend))) {
       has_depend = true;
       bool mask_out = (depend_node->inputs().size() == 3);
+      mask_out = true;
       for (size_t j = 1; j < depend_node->inputs().size(); ++j) {
         AnfNodePtr depend_input_node = depend_node->input(j);
         if (depend_input_node->isa<CNode>()) {
@@ -361,9 +362,9 @@ int AnfExporter::Anf2Fb(const FuncGraphPtr &func_graph, const std::unique_ptr<sc
     if (prim->name() == mindspore::ops::kNameDepend || prim->name() == mindspore::ops::kNameControlDepend) {
       continue;
     }
-    if (prim->name() == "make_tuple") {
-      continue;
-    }
+//    if (prim->name() == "make_tuple") {
+//      continue;
+//    }
 
     if (prim->name() == mindspore::ops::kNameTupleGetItem || prim->name() == mindspore::ops::kNameMakeTuple) {
       continue;
diff --git a/mindspore/lite/tools/optimizer/graph/redundant_op_remove_pass.cc b/mindspore/lite/tools/optimizer/graph/redundant_op_remove_pass.cc
index 7cb738c..87dde56 100644
--- a/mindspore/lite/tools/optimizer/graph/redundant_op_remove_pass.cc
+++ b/mindspore/lite/tools/optimizer/graph/redundant_op_remove_pass.cc
@@ -15,12 +15,28 @@
  */
 
 #include "tools/optimizer/graph/redundant_op_remove_pass.h"
-#include "mindspore/lite/include/errorcode.h"
+#include <memory>
+#include <vector>
+#include "include/errorcode.h"
+#include "ops/make_tuple.h"
 
 namespace mindspore::opt {
 namespace {
 constexpr size_t InputDoubleNum = 2;
 constexpr size_t InputTripleNum = 3;
+void FetchCNodeFromMakeTuple(const AnfNodePtr &anf_node, std::vector<AnfNodePtr> *inputs) {
+  MS_ASSERT(anf_node != nullptr);
+  MS_ASSERT(inputs != nullptr);
+  auto cnode = anf_node->cast<CNodePtr>();
+  if (cnode == nullptr) {
+    return;
+  }
+  for (size_t i = 1; i < cnode->size(); ++i) {
+    if (cnode->input(i)->isa<CNode>()) {
+      inputs->push_back(cnode->input(i));
+    }
+  }
+}
 }  // namespace
 int RemoveRedundantOpPass::ReplaceOp(const AnfNodePtr &anf_node, const FuncGraphManagerPtr &manager) {
   if (!utils::isa<CNodePtr>(anf_node)) {
@@ -58,6 +74,36 @@ int RemoveRedundantOpPass::ReplaceOp(const AnfNodePtr &anf_node, const FuncGraph
   return RET_OK;
 }
 
+int RemoveRedundantOpPass::ReplaceUpdateStateOp(const FuncGraphPtr &func_graph, const AnfNodePtr &anf_node) {
+  if (!utils::isa<CNodePtr>(anf_node)) {
+    MS_LOG(DEBUG) << "anf node is node a cnode.";
+    return lite::RET_NO_CHANGE;
+  }
+  auto cnode = anf_node->cast<CNodePtr>();
+  auto inputs = cnode->inputs();
+  std::vector<AnfNodePtr> new_inputs;
+  for (size_t i = 1; i < inputs.size(); ++i) {
+    if (!inputs[i]->isa<CNode>()) {
+      continue;
+    }
+    if (CheckPrimitiveType(inputs[i], prim::kPrimMakeTuple)) {
+      FetchCNodeFromMakeTuple(inputs[i], &new_inputs);
+      continue;
+    }
+    new_inputs.push_back(inputs[i]);
+  }
+  for (auto &node : new_inputs) {
+    func_graph->get_return()->add_input(node);
+  }
+  auto value = std::make_shared<UMonad>();
+  bool replace_succ = func_graph->manager()->Replace(anf_node, NewValueNode(value));
+  if (!replace_succ) {
+    MS_LOG(ERROR) << "replace redundant op failed.";
+    return lite::RET_ERROR;
+  }
+  return RET_OK;
+}
+
 int RemoveRedundantOpPass::ReplaceTupleGetItem(const AnfNodePtr &anf_node, const FuncGraphManagerPtr &manager) {
   if (!utils::isa<CNodePtr>(anf_node)) {
     MS_LOG(DEBUG) << "anf node is node a cnode.";
@@ -111,7 +157,7 @@ bool RemoveRedundantOpPass::Run(const FuncGraphPtr &func_graph) {
       status = ReplaceOp(node, manager);
     }
     if (CheckPrimitiveType(node, prim::kPrimUpdateState)) {
-      status = ReplaceOp(node, manager);
+      status = ReplaceUpdateStateOp(func_graph, node);
     }
     if (CheckPrimitiveType(node, prim::kPrimTupleGetItem)) {
       status = ReplaceTupleGetItem(node, manager);
diff --git a/mindspore/lite/tools/optimizer/graph/redundant_op_remove_pass.h b/mindspore/lite/tools/optimizer/graph/redundant_op_remove_pass.h
index 1bef786..ad7532a 100644
--- a/mindspore/lite/tools/optimizer/graph/redundant_op_remove_pass.h
+++ b/mindspore/lite/tools/optimizer/graph/redundant_op_remove_pass.h
@@ -29,6 +29,7 @@ class RemoveRedundantOpPass : public Pass {
   RemoveRedundantOpPass() : Pass("remove_redundant_op_pass") {}
   ~RemoveRedundantOpPass() override = default;
   int ReplaceOp(const AnfNodePtr &anf_node, const FuncGraphManagerPtr &manager);
+  int ReplaceUpdateStateOp(const FuncGraphPtr &func_graph, const AnfNodePtr &anf_node);
   int ReplaceTupleGetItem(const AnfNodePtr &anf_node, const FuncGraphManagerPtr &manager);
   bool Run(const FuncGraphPtr &graph) override;
 
-- 
2.7.4

